In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import SVG

import networkx as nx
from networkx.algorithms import approximation as apx
from sklearn.cluster import KMeans
from sknetwork.hierarchy import Paris, cut_balanced, cut_straight
from sknetwork.visualization import svg_dendrogram, svg_graph

from mpcstb import landscape as ld
from mpcstb import stp_parser as stp
from mpcstb.constants import DIR_DATA

In [3]:
%matplotlib inline

# parser

## load file

In [4]:
ES10FSR01_FILE = DIR_DATA / "raw" / "ES10FST" / "es10fst01.stp"

In [5]:
s = stp.SteinerTreeProblem(stp_file=ES10FSR01_FILE,
                           update_factors=[0.0, 1.0],
                           default_prize=2.0,
                           graph_period_attributes={"total_weight": [40.0, 40.0], 
                                                    "total_distance": [40.0, 40.0]}
                          )

In [6]:
s.num_periods()

2

## print data

In [7]:
from pprint import pprint
pprint([n for n in s.nodes(data=True)])

[(1, {'is_terminal': 1, 'prize': 2.0, 'x': 8183892, 'y': 4929768}),
 (2, {'is_terminal': 1, 'prize': 2.0, 'x': 4060003, 'y': 6464021}),
 (3, {'is_terminal': 1, 'prize': 2.0, 'x': 6673119, 'y': 3007983}),
 (4, {'is_terminal': 1, 'prize': 2.0, 'x': 3283856, 'y': 4356078}),
 (5, {'is_terminal': 1, 'prize': 2.0, 'x': 7079213, 'y': 7496262}),
 (6, {'is_terminal': 1, 'prize': 2.0, 'x': 3988946, 'y': 9371529}),
 (7, {'is_terminal': 1, 'prize': 2.0, 'x': 1729512, 'y': 135599}),
 (8, {'is_terminal': 1, 'prize': 2.0, 'x': 4489141, 'y': 562685}),
 (9, {'is_terminal': 1, 'prize': 2.0, 'x': 7066958, 'y': 1568812}),
 (10, {'is_terminal': 1, 'prize': 2.0, 'x': 6843121, 'y': 5391302}),
 (11, {'is_terminal': 0, 'prize': 2.0, 'x': 3283856, 'y': 562685}),
 (12, {'is_terminal': 0, 'prize': 2.0, 'x': 6673119, 'y': 1568812}),
 (13, {'is_terminal': 0, 'prize': 2.0, 'x': 3283856, 'y': 4929768}),
 (14, {'is_terminal': 0, 'prize': 2.0, 'x': 4060003, 'y': 4929768}),
 (15, {'is_terminal': 0, 'prize': 2.0, 'x': 66

In [8]:
pprint([e for e in s.edges(data=True)])

[(1, 16, {'distance': 1340771, 'weight': 1340771}),
 (2, 18, {'distance': 2783118, 'weight': 2783118}),
 (2, 17, {'distance': 71057, 'weight': 71057}),
 (2, 14, {'distance': 1534253, 'weight': 1534253}),
 (3, 15, {'distance': 1921785, 'weight': 1921785}),
 (3, 12, {'distance': 1439171, 'weight': 1439171}),
 (4, 13, {'distance': 573690, 'weight': 573690}),
 (4, 11, {'distance': 3793393, 'weight': 3793393}),
 (5, 18, {'distance': 1268333, 'weight': 1268333}),
 (6, 17, {'distance': 2907508, 'weight': 2907508}),
 (7, 11, {'distance': 1981430, 'weight': 1981430}),
 (8, 12, {'distance': 3190105, 'weight': 3190105}),
 (8, 11, {'distance': 1205285, 'weight': 1205285}),
 (9, 12, {'distance': 393839, 'weight': 393839}),
 (10, 18, {'distance': 1072719, 'weight': 1072719}),
 (10, 16, {'distance': 461534, 'weight': 461534}),
 (13, 14, {'distance': 776147, 'weight': 776147}),
 (13, 17, {'distance': 2239343, 'weight': 2239343}),
 (14, 15, {'distance': 2613116, 'weight': 2613116}),
 (15, 16, {'distanc

## get node, edges and graph attributes

In [9]:
s.node_attributes_df()

,prize,is_terminal,x,y
node,,,,
1,2.0,1,8183892,4929768
2,2.0,1,4060003,6464021
3,2.0,1,6673119,3007983
4,2.0,1,3283856,4356078
5,2.0,1,7079213,7496262
6,2.0,1,3988946,9371529
7,2.0,1,1729512,135599
8,2.0,1,4489141,562685
9,2.0,1,7066958,1568812


In [10]:
s.edge_attributes_df()

distance   weight
source target                   
1      16       1340771  1340771
2      18       2783118  2783118
       17         71057    71057
       14       1534253  1534253
3      15       1921785  1921785
       12       1439171  1439171
4      13        573690   573690
       11       3793393  3793393
5      18       1268333  1268333
6      17       2907508  2907508
7      11       1981430  1981430
8      12       3190105  3190105
       11       1205285  1205285
9      12        393839   393839
10     18       1072719  1072719
       16        461534   461534
13     14        776147   776147
       17       2239343  2239343
14     15       2613116  2613116
15     16        170002   170002

In [11]:
s.graph

{'periods': 2,
 'update_factor-01': 0.0,
 'update_factor-02': 1.0,
 'total_weight-01': 40.0,
 'total_weight-02': 40.0,
 'total_distance-01': 40.0,
 'total_distance-02': 40.0}

In [12]:
nx.get_node_attributes(s, "x")

{1: 8183892,
 2: 4060003,
 3: 6673119,
 4: 3283856,
 5: 7079213,
 6: 3988946,
 7: 1729512,
 8: 4489141,
 9: 7066958,
 10: 6843121,
 11: 3283856,
 12: 6673119,
 13: 3283856,
 14: 4060003,
 15: 6673119,
 16: 6843121,
 17: 3988946,
 18: 6843121}

# writer

## write STP file

In [13]:
s.write_stp("test.stp")

In [14]:
t = stp.SteinerTreeProblem(stp_file="test.stp",
                           update_factors=[0.0, 1.0],
                           default_prize=2.0,
                           graph_period_attributes={"total_weight": [40.0, 40.0], 
                                                    "total_distance": [40.0, 40.0]}
                          )

In [15]:
!cat test.stp

0 STP File, STP Format Version 1.0

SECTION Graph
Nodes 18
Edges 20
E 1 16 1340771
E 2 18 2783118
E 2 17 71057
E 2 14 1534253
E 3 15 1921785
E 3 12 1439171
E 4 13 573690
E 4 11 3793393
E 5 18 1268333
E 6 17 2907508
E 7 11 1981430
E 8 12 3190105
E 8 11 1205285
E 9 12 393839
E 10 18 1072719
E 10 16 461534
E 13 14 776147
E 13 17 2239343
E 14 15 2613116
E 15 16 170002
END

SECTION Terminals
Terminals 10
T 1
T 2
T 3
T 4
T 5
T 6
T 7
T 8
T 9
T 10
END

SECTION Coordinates
DD 1 8183892 4929768
DD 2 4060003 6464021
DD 3 6673119 3007983
DD 4 3283856 4356078
DD 5 7079213 7496262
DD 6 3988946 9371529
DD 7 1729512 135599
DD 8 4489141 562685
DD 9 7066958 1568812
DD 10 6843121 5391302
DD 11 3283856 562685
DD 12 6673119 1568812
DD 13 3283856 4929768
DD 14 4060003 4929768
DD 15 6673119 4929768
DD 16 6843121 4929768
DD 17 3988946 6464021
DD 18 6843121 6464021
END

EOF


In [16]:
!rm test.stp

In [17]:
s.write_graphml("test.graphml")

## write graphml file

In [18]:
!cat test.graphml

<?xml version='1.0' encoding='utf-8'?>
<graphml xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd"><key id="d12" for="edge" attr.name="distance" attr.type="int"/>
<key id="d11" for="edge" attr.name="weight" attr.type="int"/>
<key id="d10" for="node" attr.name="y" attr.type="int"/>
<key id="d9" for="node" attr.name="x" attr.type="int"/>
<key id="d8" for="node" attr.name="is_terminal" attr.type="int"/>
<key id="d7" for="node" attr.name="prize" attr.type="double"/>
<key id="d6" for="graph" attr.name="total_distance-02" attr.type="double"/>
<key id="d5" for="graph" attr.name="total_distance-01" attr.type="double"/>
<key id="d4" for="graph" attr.name="total_weight-02" attr.type="double"/>
<key id="d3" for="graph" attr.name="total_weight-01" attr.type="double"/>
<key id="d2" for="graph" attr.name="update_factor-02" attr.type="double

In [19]:
!rm test.graphml

# Watermark

In [20]:
%load_ext watermark

In [21]:
# NBVAL_IGNORE_OUTPUT
%watermark -a "Patricio Reyes" -u -d -t -v -m -g -wb

Patricio Reyes 
last updated: 2020-10-20 12:12:26 

CPython 3.8.6
IPython 7.18.1

compiler   : GCC 7.5.0
system     : Linux
release    : 4.4.0-193-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 40
interpreter: 64bit
Git hash   : f028298827dfc0bc82e5941882a6bd14491dc9b3
Git branch : master
watermark 2.0.2


In [22]:
# NBVAL_IGNORE_OUTPUT
%watermark -iv

networkx 2.5
pandas   1.1.3
json     2.0.9
numpy    1.19.2

